In [1]:
portfolio_sd_t <- function(sd, cov, weight){
  stopifnot(length(sd) > 1, is.matrix(cov), length(sd) == length(weight),
            sum(weight) == 1)
  for(i in 1:(length(sd)-1)){
    for(j in (i+1):length(sd)){
      n <- sum(weight[i:length(sd)]^2*sd[i:length(sd)]^2)
      m <- sum(2*weight[i]*weight[j]*cov[i,j])
      sd <- sqrt(sum(weight[i:length(sd)]^2*sd[i:length(sd)]^2+
                       2*weight[i]*weight[j]*cov[i,j]))
      return(sd)
    }
  }
}
portfolio_return <- function(returns, weight){
  stopifnot(sum(weight) == 1)
  if(!is.vector(returns) > 1){
    mean <- matrix(0, nrow = ncol(returns), ncol = 1)
    for(i in 1:ncol(returns)){
      mean[i] <- mean(as.matrix(returns[,i]), na.rm = T)
    }
  } else{
    if(is.vector(returns)){
      mean <- returns
    }
  }
  total_return <- sum(weight*mean)
  return(total_return)
}

In [2]:
pacotes <- c("tidyquant", "tidyr", "dplyr","stats")
options(warn = -1)
for(i in pacotes){
    library(i, character.only = T)
}

Loading required package: lubridate


Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union


Loading required package: PerformanceAnalytics

Loading required package: xts

Loading required package: zoo


Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric



Attaching package: 'PerformanceAnalytics'


The following object is masked from 'package:graphics':

    legend


Loading required package: quantmod

Loading required package: TTR

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Version 0.4-0 included new data defaults. See ?getSymbols.

== Need to Learn tidyquant? ====================================================
Business Science offers a 1-hour course - Learning Lab #9: Performance Analysis & Portfolio Optimization with tidyquant!
</> Learn more at: https://university.business-science.io/p/learning-

In [5]:
acoes <- c("TRPL4.SA","ITSA4.SA","ITUB4.SA","EGIE3.SA","CPLE3.SA",
           "LREN3.SA","HYPE3.SA","B3SA3.SA","VVAR3.SA","ABCB4.SA",
          "SUZB3.SA","BEES3.SA","ABEV3.SA","FLRY3.SA")
retornos_d <- tq_get(acoes, from = "2019-07-25", to = "2020-07-25") %>%
  group_by(symbol) %>%
  tq_transmute(select = close, mutate_fun= periodReturn, period = "daily")
retornos_a <- tq_get(acoes, from = "2014-01-01", to = "2020-07-25") %>%
  group_by(symbol) %>%
  tq_transmute(select = close, mutate_fun= periodReturn, period = "yearly") %>%
  mutate(year = year(date)) %>%
  group_by(year,symbol) %>%
  summarise(yearly.returns) %>%
  spread(key = "symbol", value = "yearly.returns")

`summarise()` regrouping output by 'year' (override with `.groups` argument)



In [6]:
sd_d <- retornos_d %>%
  group_by(symbol) %>%
  summarise(sd = sd(daily.returns)) %>%
  spread(key = "symbol", value = sd)
cov <- retornos_d %>%
  spread(key = "symbol", value = daily.returns)
cov$date <- NULL
cov <- cov(cov)

`summarise()` ungrouping output (override with `.groups` argument)



In [10]:
pesos <- c(0.06,0.02,0.06,0.04,0.12,0.13,0.02,0.07,0.13,0.03,0.12,0.04,0.1,0.06)

In [11]:
sd_cart <- portfolio_sd_t(sd = sd_d, cov = cov, weight = pesos)*sqrt(252)
retorno_esp <- portfolio_return(returns = retornos_a[2:ncol(retornos_a)], weight = pesos)
paste0("Desvio-Anual =", round(sd_cart*100,3), "%  Retorno Esperado =", round(retorno_esp*100,3),"%")
sharpe <- (retorno_esp-0.0225)/sd_cart
names(sharpe) <- "Sharpe da Carteira"
round(sharpe,3)

[1] "Desvio-Anual =16.008%  Retorno Esperado =19.292%"

Sharpe da Carteira 
             1.065

In [12]:
sd_cart
retorno_esp

[1] 0.1600811

[1] 0.1929225

In [ ]:
#Minha carteira tem beta de 0.8239262828